In [1]:
import pandas as pd
import nimfa
import scipy
from timeit import default_timer as timer

from sklearn.cluster import KMeans

from hawkes.cluster import FormatAndSplit, ToSparse, GetClosestCustomerCluster
from hawkes.utils import GetTimeSeriesFromDF, GetInfluenceMatrix
from hawkes.hawkes import ExpectationMaximization, Q

%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('/Users/arnaud/cellule/data/bnpp/ETSAnonymousPricesFull.csv')

/Users/arnaud/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (9,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_df, train_coo, test_df, test_coo = FormatAndSplit(df, cut_date=pd.to_datetime('20131106', format='%Y%m%d'))

In [26]:
#### TRAIN

# Clustering model
cust = GetClosestCustomerCluster('GAZPRU', train_coo, train_df, M_clusters=10, N_clusters=7)
print "Cluster closest to GAZPRU: %s" % cust

idx = [c in cust for c in train_df['Customer']]
dd = train_df.ix[idx, :]
print "Train set after filtering: %s" % str(dd.shape)

# Leader-Follower model
train_time, train_mark = GetTimeSeriesFromDF(dd)
mu, a, b, p = ExpectationMaximization(train_time, niter=100)

customer_ids = train_mark["Customer"].values
g, unique_ids = GetInfluenceMatrix(p, customer_ids)

Cluster closest to GAZPRU: set([385, 523, 269, 656, 145, 1299, 791, 1314, 1069, 826, 151, 1085, 960, 69, 967, 208, 469, 348, 864, 1303, 997, 747, 1260, 111])
Train set after filtering: (4881, 58)


In [29]:
#### TEST

idx = [c in cust for c in test_df['Customer']]
dd = test_df.ix[idx, :]
print "Test set after filtering: %s" % str(dd.shape)

test_time, test_mark = GetTimeSeriesFromDF(dd)

gg = g.copy()
threshold = 0.
gg[gg < threshold] = 0
print "Number of strong inluencers: %s" % sum(gg > 0)

influencer_dict = dict((unique_ids[i], unique_ids[l]) for i, l in enumerate(scipy.sparse.lil_matrix(gg).rows) if l != [])

customer_ids_test = test_mark["Customer"].values

tpr = 0
fpr = 0
k = 10
m = 0
for influencer, influencee in influencer_dict.iteritems():
    for i in np.where(customer_ids_test == influencer)[0]:
        m += 1
        tpr += sum(j in influencee for j in customer_ids_test[i: i + k]) / float(k)
        fpr += (k - sum(j in influencee for j in customer_ids_test[i: i + k])) / float(k)

tpr /= m
fpr /= m

print "tpr = %.3f" % tpr
print "fpr = %.3f" % fpr

Test set after filtering: (8733, 58)
Number of strong inluencers: 495
tpr = 0.999
fpr = 0.001
